In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from tensorflow.keras import activations
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg19 import VGG19

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

## Image Preprocessing
- ### split the provided training set in 85% for actual training set and 15% to be used as validation set

In [3]:
SEED = 1849
image_size = (256, 256)
class_names=['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'InsideCity', 'Kitchen', 'LivingRoom', 'Mountain', 'Office', 'OpenCountry', 'Store', 'Street', 'Suburb', 'TallBuilding']
num_classes = len(class_names)

train_ds = keras.preprocessing.image_dataset_from_directory(
    directory="dataset/train",
    color_mode="rgb",
    class_names=class_names,
    validation_split=0.15,
    subset="training",
    shuffle=True,
    seed=SEED,
    image_size=image_size,
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    directory="dataset/train",
    color_mode="rgb",
    class_names=class_names,
    validation_split=0.15,
    subset="validation",
    shuffle=True,
    seed=SEED,
    image_size=image_size,
)

test_ds = keras.preprocessing.image_dataset_from_directory(
    directory="dataset/test",
    color_mode="rgb",
    class_names=class_names,
    validation_split=None,
    subset=None,
    shuffle=True,
    seed=SEED,
    image_size=image_size,
)

Found 1500 files belonging to 15 classes.
Using 1275 files for training.
Found 1500 files belonging to 15 classes.
Using 225 files for validation.
Found 2985 files belonging to 15 classes.


## Data Augmentation of the training set
- ### left-to-right reflection
- ### random crop (cropping a patch of 240 x 240)
- ### random rotation
- ### resizing of the image (224 x 224)

In [4]:
IMG_SIZE = 224
CROP_SIZE = 240

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal", seed=SEED),
        layers.experimental.preprocessing.RandomCrop(height=CROP_SIZE, width=CROP_SIZE, seed=SEED),
        layers.experimental.preprocessing.RandomRotation(factor=0.1, seed=SEED),
        layers.experimental.preprocessing.Resizing(height=IMG_SIZE, width=IMG_SIZE)
    ]
)

data_resizing = keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(height=IMG_SIZE, width=IMG_SIZE)
    ]
)

In [5]:
#just a preprocessing needed for the VGG19 model
training_dataset_VGG19 = train_ds.map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y))
validation_dataset_VGG19 = val_ds.map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y))
test_dataset_VGG19 = test_ds.map(lambda x, y: (tf.keras.applications.vgg19.preprocess_input(x), y))

In [6]:
augmented_training_dataset = training_dataset_VGG19.map(lambda x, y: (data_augmentation(x), y))
resized_training_dataset = training_dataset_VGG19.map(lambda x, y: (data_resizing(x), y))

validation_dataset = validation_dataset_VGG19.map(lambda x, y: (data_resizing(x), y))
test_dataset = test_dataset_VGG19.map(lambda x, y: (data_resizing(x), y))

In [7]:
training_dataset = augmented_training_dataset.concatenate(resized_training_dataset)

## VGG19 loading...

In [8]:
VGG19_model = VGG19(weights='imagenet', input_shape=(224,224,3), include_top=True)
VGG19_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Let's remove the last layer and add the new final dense layer manually

In [9]:
from tensorflow.keras import models

bias_init = initializers.Zeros()
weight_init = initializers.RandomNormal(mean=0, stddev=0.01)

model = models.Sequential()

for layer in VGG19_model.layers[:-1]:
	model.add(layer)

model.add(layers.Dense(units=num_classes, bias_initializer=bias_init, kernel_initializer=weight_init, activation="softmax"))

## Now freeze all the layers except the last

In [10]:
for layer in model.layers[:-1]:
    layer.trainable = False

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

## Starting the fine-tuning..

In [11]:
opt = optimizers.Adam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3,
restore_best_weights=True)

minibatches = 32
epochs = 5

history = model.fit(training_dataset, validation_data=validation_dataset, batch_size=minibatches, epochs=epochs,
callbacks=[early_stopping], shuffle=True, verbose=1)

Epoch 1/5
80/80 [==============================] - 359s 4s/step - loss: 1.1675 - accuracy: 0.6002 - val_loss: 0.4818 - val_accuracy: 0.8267
Epoch 2/5
80/80 [==============================] - 367s 5s/step - loss: 0.3236 - accuracy: 0.8862 - val_loss: 0.4197 - val_accuracy: 0.8267
Epoch 3/5
80/80 [==============================] - 369s 5s/step - loss: 0.2503 - accuracy: 0.9128 - val_loss: 0.4155 - val_accuracy: 0.8533
Epoch 4/5
80/80 [==============================] - 366s 5s/step - loss: 0.2159 - accuracy: 0.9213 - val_loss: 0.3907 - val_accuracy: 0.8756
Epoch 5/5
80/80 [==============================] - 367s 5s/step - loss: 0.1823 - accuracy: 0.9399 - val_loss: 0.4495 - val_accuracy: 0.8578


In [20]:
test_eval = model.evaluate(test_dataset, verbose=0)
print('Overall Test accuracy: {number:.{digits}f}'.format(number=test_eval[1], digits=2))

Overall Test accuracy: 0.88


## Let's employ the VGG19 as a feature extractor for the SVM

In [12]:
feature_extractor_model = VGG19(weights='imagenet', input_shape=(224,224,3), include_top=False)
feature_extractor_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
test_list = list(test_dataset) #casting necessary due to the randomized iteration over the datasets
test_labels = np.concatenate([y for x, y in test_list], axis=0)
test_images = np.concatenate([x for x, y in test_list], axis=0)

training_list = list(training_dataset)
training_labels = np.concatenate([y for x, y in training_list], axis=0)
training_images = np.concatenate([x for x, y in training_list], axis=0)

In [14]:
features = feature_extractor_model.predict(training_images)

## Just flattening the feature dimensions

In [15]:
nsamples, x, y, z = features.shape
features_reshaped = features.reshape(nsamples, x*y*z)
features_reshaped.shape

(2550, 25088)

## Train the linear SVM

In [16]:
from sklearn import svm

linear_clf = svm.SVC(kernel='linear', C=1, decision_function_shape='ovo').fit(features_reshaped, training_labels)

## Train the non-linear SVM

In [21]:
clf = svm.SVC(gamma='auto', C=1, decision_function_shape='ovo').fit(features_reshaped, training_labels)

## Extract features from the test dataset

In [17]:
from sklearn import metrics

test_features = feature_extractor_model.predict(test_images)
nsamples, x, y, z = test_features.shape
test_features_reshaped = test_features.reshape(nsamples, x*y*z)

## Evaluate the linear SVM on the test dataset

In [18]:
y_pred = linear_clf.predict(features_reshaped)
print("Training Accuracy:",metrics.accuracy_score(training_labels, y_pred))

Training Accuracy: 1.0


In [19]:
y_pred = linear_clf.predict(test_features_reshaped)
print("Test Accuracy:",metrics.accuracy_score(test_labels, y_pred))

Test Accuracy: 0.8954773869346734


## Evaluate the non-linear SVM on the test dataset

In [22]:
y_pred = clf.predict(features_reshaped)
print("Training Accuracy:",metrics.accuracy_score(training_labels, y_pred))

Training Accuracy: 1.0


In [23]:
y_pred = clf.predict(test_features_reshaped)
print("Test Accuracy:",metrics.accuracy_score(test_labels, y_pred))

Test Accuracy: 0.06834170854271357


## Discussion on SVM

As we can see, the 2 SVMs overfit (a lot), but the linear one have a good test accuracy!

The non linear SVM instead is worse then a random classifier! 

(random classifier -> 7% accuracy)